In [ ]:
#meta 7/4/2021 HF Transformers Course - Ch.2 Using HF Transfomers: Behind the Pipeline
#src: https://huggingface.co/course/chapter2/2?fw=tf

#history
#7/4/2021 Ch.2 Behind the Pipeline
#      same notebook, in Colab (much faster)
#      


# Chapter 2

- How to use tokenizers and models to replicate the `pipeline` API’s behavior  
- How to load and save models and tokenizers  
- Different tokenization approaches, such as word-based, character-based, and subword-based  
- How to handle multiple sentences of varying lengths  

# Behind the pipeline (TensorFlow)




##### Setup
Install the Transformers and Datasets libraries to run this notebook.

In [1]:
! pip install datasets transformers[sentencepiece]

     |████████████████████████████████| 245kB 4.3MB/s 
     |████████████████████████████████| 2.5MB 31.5MB/s 
     |████████████████████████████████| 245kB 37.5MB/s 
     |████████████████████████████████| 122kB 20.2MB/s 
     |████████████████████████████████| 901kB 37.5MB/s 
     |████████████████████████████████| 3.3MB 39.0MB/s 
     |████████████████████████████████| 1.1MB 40.9MB/s 
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


Chapter 1 complete example: 
this pipeline groups together three steps: preprocessing, passing the inputs through the model, and postprocessing  
![NLP pipeline]("img/nlp_pipeline.png")

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier([
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
])

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

## 1. Preprocessing with a tokenizer
convert the text inputs into numbers

All this preprocessing needs to be done in exactly the same way as when the model was pretrained, so we first need to download that information from the Model Hub. To do this, we use the `AutoTokenizer` class and its `from_pretrained` method. Using the checkpoint name of our model, it will automatically fetch the data associated with the model’s tokenizer and cache it.

The default checkpoint of the sentiment-analysis pipeline is `distilbert-base-uncased-finetuned-sst-2-english` (see its model card https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)



In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Once we have the tokenizer, we can directly pass our sentences to it and we’ll get back a dictionary that’s ready to feed to our model! The only thing left to do is to convert the list of input IDs to tensors.

In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")

#view results as TensorFlow tensors:
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


The output itself is a dictionary containing two keys, `input_ids` and `attention_mask`. `input_ids` contains two rows of integers (one for each sentence) that are the unique identifiers of the tokens in each sentence.

## 2. Going through the model
Download our pretrained model the same way we did with our tokenizer. 🤗 Transformers provides an `TFAutoModel` class which also has a `from_pretrained` method:

In [5]:
#download the same checkpoint we used in our pipeline before and instantiated a model with it
from transformers import TFAutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['dropout_19', 'classifier', 'pre_classifier']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


This architecture contains only the base Transformer module: given some inputs, it outputs what we’ll call hidden states, also known as features. For each model input, we’ll retrieve a high-dimensional vector representing the contextual understanding of that input by the Transformer model.

While these hidden states can be useful on their own, they’re usually inputs to another part of the model, known as the head. In Chapter 1, the different tasks could have been performed with the same architecture, but each of these tasks will have a different head associated with it.

##### A high-dimensional vector?

The vector output by the Transformer module is usually large. It generally has three dimensions:

1. Batch size: The number of sequences processed at a time (2 in our example).  
2. Sequence length: The length of the numerical representation of the sequence (16 in our example).  
3. Hidden size: The vector dimension of each model input.

It is said to be “high dimensional” because of the last value. The hidden size can be very large (768 is common for smaller models, and in larger models this can reach 3072 or more).

In [6]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 16, 768)


The output of the Transformer model is sent directly to the model head to be processed.

The embeddings layer converts each input ID in the tokenized input into a vector that represents the associated token. The subsequent layers manipulate those vectors using the attention mechanism to produce the final representation of the sentences.

There are many different architectures available in 🤗 Transformers, with each one designed around tackling a specific task. 

In [7]:
#this example:  model with a sequence classification head (to be able to classify the sentences as positive or negative). 
#So won’t actually use the TFAutoModel class, but TFAutoModelForSequenceClassification:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_38']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now if we look at the shape of our inputs, the dimensionality will be much lower: the model head takes as input the high-dimensional vectors and outputs vectors containing two values (one per label):

In [8]:
print(outputs.logits.shape)

(2, 2)


## 3. Postprocessing the output

The values we get as output from our model don’t necessarily make sense by themselves.

In [9]:
print(outputs.logits)

tf.Tensor(
[[-1.5606964  1.6122813]
 [ 4.169231  -3.3464475]], shape=(2, 2), dtype=float32)


Those are not probabilities but `logits`, the raw, unnormalized scores outputted by the last layer of the model. To be converted to probabilities, they need to go through a [SoftMax](https://en.wikipedia.org/wiki/Softmax_function) layer (all 🤗 Transformers models output the logits, as the loss function for training will generally fuse the last activation function, such as `SoftMax`, with the actual loss function, such as `cross entropy`):

In [10]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[4.0195383e-02 9.5980465e-01]
 [9.9945587e-01 5.4418424e-04]], shape=(2, 2), dtype=float32)


These are recognizable probability scores.

To get the labels corresponding to each position, we can inspect the `id2label` attribute of the model config:

In [11]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

Now we can conclude that the model predicted the following:

First sentence: NEGATIVE: 0.0402, POSITIVE: 0.9598 
Second sentence: NEGATIVE: 0.9995, POSITIVE: 0.0005 

We have successfully reproduced the three steps of the pipeline:  
1. preprocessing with tokenizers   
2. passing the inputs through the model    
3. postprocessing

and obtained the same results!